In [1]:
import cv2
import numpy as np

In [2]:
help(cv2.goodFeaturesToTrack)

Help on built-in function goodFeaturesToTrack:

goodFeaturesToTrack(...)
    goodFeaturesToTrack(image, maxCorners, qualityLevel, minDistance[, corners[, mask[, blockSize[, useHarrisDetector[, k]]]]]) -> corners
    .   @brief Determines strong corners on an image.
    .   
    .   The function finds the most prominent corners in the image or in the specified image region, as
    .   described in @cite Shi94
    .   
    .   -   Function calculates the corner quality measure at every source image pixel using the
    .       #cornerMinEigenVal or #cornerHarris .
    .   -   Function performs a non-maximum suppression (the local maximums in *3 x 3* neighborhood are
    .       retained).
    .   -   The corners with the minimal eigenvalue less than
    .       \f$\texttt{qualityLevel} \cdot \max_{x,y} qualityMeasureMap(x,y)\f$ are rejected.
    .   -   The remaining corners are sorted by the quality measure in the descending order.
    .   -   Function throws away each corner for which t

In [3]:
help(cv2.calcOpticalFlowPyrLK)

Help on built-in function calcOpticalFlowPyrLK:

calcOpticalFlowPyrLK(...)
    calcOpticalFlowPyrLK(prevImg, nextImg, prevPts, nextPts[, status[, err[, winSize[, maxLevel[, criteria[, flags[, minEigThreshold]]]]]]]) -> nextPts, status, err
    .   @brief Calculates an optical flow for a sparse feature set using the iterative Lucas-Kanade method with
    .   pyramids.
    .   
    .   @param prevImg first 8-bit input image or pyramid constructed by buildOpticalFlowPyramid.
    .   @param nextImg second input image or pyramid of the same size and the same type as prevImg.
    .   @param prevPts vector of 2D points for which the flow needs to be found; point coordinates must be
    .   single-precision floating-point numbers.
    .   @param nextPts output vector of 2D points (with single-precision floating-point coordinates)
    .   containing the calculated new positions of input features in the second image; when
    .   OPTFLOW_USE_INITIAL_FLOW flag is passed, the vector must have the 

In [18]:
import cv2
import numpy as np

corner_track_param = dict(maxCorners = 10, 
                          qualityLevel = 0.2, 
                          minDistance = 10, 
                          blockSize = 7)

lk_param = dict(winSize = (200, 200), 
                maxLevel = 2, 
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

color = np.random.randint(0, 255, (100, 3))

cap = cv2.VideoCapture(r'C:\Users\Data\computer vision\video\Slow Traffic Small Open CV.mp4')

ret, prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask =None, **corner_track_param)
mask = np.zeros_like(prev_frame)


while(1):
    ret, frame = cap.read()
    
    if not ret:
        print('Not Frame')
        break
    next_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, next_gray,prevPts, None, **lk_param)
   
    
    
    good_new = nextPts[status ==1]
    good_prev = prevPts[status ==1]
    
    
    
    
    for i,(new, prev) in enumerate(zip(good_new, good_prev)):
        x_new, y_new = np.intp(new.ravel())
        x_prev, y_prev = np.intp(prev.ravel())
        
        mask = cv2.line(mask, (x_new, y_new), (x_prev, y_prev), color[i].tolist(), 4)
        frame = cv2.circle(frame, (x_new, y_new), 6, color[i].tolist(), -1)
        
        img = cv2.add(mask , frame)
        cv2.imshow('frame', img)
        
        k = cv2.waitKey(10) & 0xff
        if k == 27:
            break
            
        prev_gray = next_gray.copy()
        prevPts = good_new.reshape(-1,1,2)
        
cap.release()
cv2.destroyAllWindows()
    

Not Frame
